In [0]:
''' Scraping Libraries '''
from bs4 import BeautifulSoup as Soup 
import pandas as pd
import requests

# General Scraping Functions

In [0]:
headers = {'User-Agent':'Edge'}

def parse_row(rows):
    return [str(x.string) for x in rows.find_all('td')]

def sewp(url):
    r = requests.get(url, headers = headers)
    html = Soup(r.text, features = 'lxml')
    return html

def numeric(frame):
    try:
        frame[[i]] = frame[[i]].astype(float)
    except:
        pass

def find_links(url):
    html = sewp(url)
    href_tags = html.find_all(href = True)
    href_tags = list(href_tags)
    hrefs = [tag.get('href') for tag in href_tags]
    return hrefs

# Baseball Reference Specific Crawling functions

In [0]:
''' Baseball Reference Specific Functions '''

def find_data(url):
    html = sewp(url)
    rws = html.find_all('tr')
    # Filtering out colligate and NWDS rows
    data = []
    for row in rws:
        row = parse_row(row)
        if 'NWDS' in row or 'NAIA' in row or 'NCAA' in row:
            data.append(row)
    df = pd.DataFrame(data)
    return df

''' Crawling Function '''
def crawl(_id):
    league_links = find_links('https://www.baseball-reference.com/register/league.cgi?id=' + _id)

    # Find team links
    team_links = []
    for href in league_links:
          if '/register/team.cgi?' in href:
            team_links.append(href)
     
    # scrape the team links for the player links
    links_from_team = []
    for href in team_links:
          links_from_team.append(find_links('https://www.baseball-reference.com' + href))
    
    # Append relevant links to list
    player_links = []
    for href in links_from_team:
          for link in href:
            if '/register/player.fcgi?id=' in link:
                  player_links.append(link)
    
    # finish by Returning the player data to a list
    player_data = []
    for link in player_links:
          player_data.append(find_data('https://www.baseball-reference.com' + link))
    
    for df in player_data:
          numeric(df)

    return player_data

# Collecting Data

In [0]:
''' Dictionary of league id's from each year we want to crawl '''
nwl_yearid_dict = {2021:'f5c87b08',2020:'78f2935d',2019:'817f5f93',2018:'6a2b88b5',
                   2017:'c290e2ac',2016:'b33681e2',2015:'1671dc07'}

# Gets all years
def get_league(league_yearid_dict):
    all_players = []
    for year in league_yearid_dict.values():
        all_players.append(crawl(year))
    return all_players

# Use this function to see the amount of data we scraped
def check(list_of_league_data, league_name):
    players = 0
    for i in list_of_league_data:
        num_players = len(i)
        players += num_players
    print(f'Evaluation for {league_name}:')
    print(f'Number of years: {len(list_of_league_data)}')
    print(f'Total number of players: {players}')

In [0]:
NWL = get_league(nwl_yearid_dict)
print(check(NWL, 'Northwoods League'))

# Cleaning Data

In [0]:
''' Adding Columns '''
batting_cols = ['age','age-diff','tm','lg','lev','aff','g','pa','ab','r','h','2b','3b',
               'hr','rbi','sb','cs','bb','so','ba','obp','slg','ops','tb','gdp','hbp','sh','sf','ibb']
pitching_cols = ['age','age-diff','tm','lg','lev','aff','w','l','w-l','era','ra9','g','gs','gf',
                 'cg','sho','sv','ip','h','r','er','hr','bb','ibb','so','hbp','bk','wp','bf','whip','h9',
                 'hr9','bb9','so9','so/bb']

# Add columns
for lst in NWL:
    for df in lst:
        if len(df.columns) == 29:
            df.columns = batting_cols
            df['POS'] = 'Batter'
        elif len(df.columns) == 35:
            df.columns = pitching_cols
            df['POS'] = 'Pitcher'

In [ ]:
''' Organize '''
# for some reason index 0 returns an empty list...
_1 = NWL[1]
_2 = NWL[2]
_3 = NWL[3]
_4 = NWL[4]
_5 = NWL[5]
_6 = NWL[6]

# finding total number of players (Dataframes)
lodfs = [_1, _2, _3, _4, _5, _6]
tot_len = 0
for x in lodfs:
    tot_len += len(x)

# using the total number to add a unique number to each player
# (doing this b/c we do not have player names) concat on id column created
x = 0
while x <= tot_len:
    for df in lodfs:
        for player in df:
            player['id'] = x
            x += 1 

# Concat based on batters and pitchers
def filt(yr):
    conc = pd.concat(yr, ignore_index = True)
    bat = conc[conc['POS'] == 'Batter']
    pitch = conc[conc['POS'] == 'Pitcher']
    return bat, pitch

# Seperating hitters and pitchers
bat_1, pitch_1 = filt(_1)
bat_22, pitch_2 = filt(_2)
bat_3, pitch_3 = filt(_3)
bat_4, pitch_4 = filt(_4)
bat_5, pitch_5 = filt(_5)
bat_6, pitch_6 = filt(_6)

In [ ]:
# seperating colligate and nwds
def align(bat, pitch):
    naia_bat = []
    ncaa_bat = []
    nwds_bat = []
    nwds_pitch = []
    ncaa_pitch = []
    naia_pitch = []
    # append batting
    nwds_bat.append(bat[bat['lev'] == 'Smr'])
    ncaa_bat.append(bat[bat['lev'] == 'NCAA'])
    naia_bat.append(bat[bat['lev'] == 'NAIA'])
    # append pitching
    nwds_pitch.append(pitch[pitch['lev'] == 'Smr'])
    ncaa_pitch.append(pitch[pitch['lev'] == 'NCAA'])
    naia_pitch.append(pitch[pitch['lev'] == 'NAIA'])
    # concat
    naiabat = pd.concat(naia_bat, ignore_index= True)
    ncaabat = pd.concat(ncaa_bat, ignore_index= True)
    nwdsbat = pd.concat(nwds_bat, ignore_index= True)
    naia_pitch = pd.concat(naia_pitch, ignore_index= True)
    ncaa_pitch = pd.concat(ncaa_pitch, ignore_index= True)
    nwds_pitch = pd.concat(nwds_pitch, ignore_index= True)
    # merge batting
    x = pd.merge(ncaabat, nwdsbat, on='id')
    y = pd.merge(naiabat, nwdsbat, on='id')
    lyst = (x,y)
    bat = pd.concat(lyst, ignore_index=True)
    # merge pitching
    c = pd.merge(ncaa_pitch, nwds_pitch, on='id')
    b = pd.merge(naia_pitch, nwds_pitch, on = 'id')
    lst = (c,b)
    pitch = pd.concat(lst, ignore_index= True)
    return bat, pitch

bat1, pitch1 = align(bat_1, pitch_1)
bat2, pitch2 = align(bat_22, pitch_2)
bat3, pitch3 = align(bat_3, pitch_3)
bat4, pitch4 = align(bat_4, pitch_4)
bat5, pitch5 = align(bat_5, pitch_5)
bat6, pitch6 = align(bat_6, pitch_6)

bat_list = (bat1, bat2, bat3, bat4, bat5, bat6)
pit_list = (pitch1, pitch2, pitch3, pitch4, pitch5, pitch6)

# Download these dataframes into excel
pitch = pd.concat(pit_list, ignore_index= True)
bat = pd.concat(bat_list, ignore_index= True)

pitch.to_excel('/Users/jense/Downloads/crawled_pitch_nwds.xlsx', index = False)
bat.to_excel('/Users/jense/Downloads/crawled_bat_nwds.xlsx', index = False)